In [ ]:
import time
import numpy as np
import heapq
from ipywidgets import interact, FloatSlider, VBox, HTML
from grid.robot.aerial.airgen_drone import AirGenDrone
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# =============================== Lidar и Скорость дрона ===============================

# Класс для управления скоростью дрона
class Velocity:
    def __init__(self, x_vel=0.0, y_vel=0.0, z_vel=0.0):
        self.x_vel = x_vel
        self.y_vel = y_vel
        self.z_vel = z_vel

# Симуляция LiDAR для обнаружения препятствий
def simulated_lidar_scan(position, radius=5.0):
    obstacles = [
        (2, 2, 1),
        (4, 4, 1)
    ]
    return [obs for obs in obstacles if np.linalg.norm(np.subtract(obs, position)) <= radius]

In [ ]:
# =============================== Алгоритм поиска пути A* ===============================

# Алгоритм A* для поиска пути на 2D-сетке с динамическими препятствиями
def astar_dynamic(start, goal, obstacles, bounds=(-10, 10)):
    def neighbors(node):
        x, y = node
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            if bounds[0] <= nx <= bounds[1] and bounds[0] <= ny <= bounds[1]:
                yield (nx, ny)

    def heuristic(a, b):
        return np.linalg.norm(np.subtract(a, b))

    start = tuple(map(int, start[:2]))
    goal = tuple(map(int, goal[:2]))
    obs_set = {tuple(map(int, o[:2])) for o in obstacles}

    frontier = [(0, start)]
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)
        if current == goal:
            break
        for next_node in neighbors(current):
            if next_node in obs_set:
                continue
            new_cost = cost_so_far[current] + 1
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(goal, next_node)
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = current

    node = goal
    path = []
    while node:
        path.append((node[0], node[1], start_pos[2]))  # фиксируем высоту
        node = came_from.get(node)
    return path[::-1] if path else []

In [ ]:
# =============================== Обучение модели ML (пример) ===============================

# Пример данных для обучения (замените на свои данные)
data = np.random.rand(100, 3)  # Пример входных данных (координаты)
labels = np.random.randint(0, 2, size=100)  # Пример меток (безопасный маршрут или нет)

# Создание модели
model = Sequential([
    Dense(64, activation='relu', input_shape=(3,)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10


1/4 ━━━━━━━━━━━━━━━━━━━━ 2s 791ms/step - accuracy: 0.5938 - loss: 0.6933

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5464 - loss: 0.6944  


Epoch 2/10


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6250 - loss: 0.6778

...

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 0.6823

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5818 - loss: 0.6748 


Epoch 10/10


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 0.6844

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5796 - loss: 0.6751 


In [ ]:
# =============================== Интерфейс пользователя ===============================

# Глобальные переменные для стартовой и целевой позиции
start_pos = [0.0, 0.0, 1.0]
target_pos = [5.0, 5.0, 3.0]

# Интерфейс для ввода целевой точки
@interact(
    x=FloatSlider(min=-10, max=10, step=1, value=5, description='X:'),
    y=FloatSlider(min=-10, max=10, step=1, value=5, description='Y:'),
    z=FloatSlider(min=0, max=10, step=1, value=3, description='Z:')
)
def set_target(x, y, z):
    global target_pos
    target_pos = [x, y, z]
    print(f"Целевая точка установлена: {target_pos}")

Целевая точка установлена: [5.0, 5.0, 3.0]


interactive(children=(FloatSlider(value=5.0, description='X:', max=10.0, min=-10.0, step=1.0), FloatSlider(val…

In [ ]:
# =============================== Управление дроном и командами ===============================

# Инициализация дрона
drone = AirGenDrone()

# Взлет дрона
def takeoff():
    print("Взлёт...")
    drone.takeoff()
    time.sleep(1)

# Посадка дрона
def land():
    print("Посадка...")
    drone.land()
    time.sleep(1)

# Перемещение по маршруту с динамическим обновлением
def fly_to_point_dynamic(path, speed=1.0):
    print("Начинаем движение по маршруту...")
    prev = path[0]
    for point in path[1:]:
        # Проверка на наличие препятствий в текущей позиции
        obstacles = simulated_lidar_scan(prev)
        if obstacles:
            print(f"🛑 Обнаружено препятствие: {obstacles}. Пересчитываю маршрут...")
            # Пересчет маршрута с учетом нового препятствия
            path = astar_dynamic(prev, target_pos, obstacles)
            if not path:
                print("❌ Не удалось построить новый маршрут. Попробуй другую цель.")
                land()
                return
            prev = path[0]  # Обновляем текущую позицию

        # Движение к следующей точке
        dx = point[0] - prev[0]
        dy = point[1] - prev[1]
        dz = point[2] - prev[2]

        norm = np.linalg.norm([dx, dy, dz])
        if norm == 0:
            continue

        vx = speed * dx / norm
        vy = speed * dy / norm
        vz = speed * dz / norm

        vel = Velocity(x_vel=vx, y_vel=vy, z_vel=vz)
        drone.moveByVelocity(vel, duration=1, frame='ned', blocking=True)
        print(f"➡ Движение к точке: {point}")
        prev = point
        time.sleep(0.5)

# Основная функция для выполнения миссии
def fly_mission():
    takeoff()
    print("🔍 Сканирование окружения...")
    obstacles = simulated_lidar_scan(start_pos)
    print(f"🧱 Обнаружены препятствия: {obstacles}")

    print("📡 Построение безопасного маршрута...")
    path = astar_dynamic(start_pos, target_pos, obstacles)
    if not path:
        print("❌ Не удалось построить маршрут. Попробуй другую цель.")
        land()
        return

    fly_to_point_dynamic(path)
    land()


Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [ ]:
# =============================== Состояние БПЛА ===============================

# Функция для отправки данных о состоянии БПЛА в GRID
def send_drone_status_to_grid(drone):
    # Получение данных о состоянии БПЛА
    position = drone.get_position()  # GPS-координаты
    velocity = drone.get_velocity()  # Скорость
    altitude = drone.get_altitude()  # Высота
    battery = drone.get_battery()    # Уровень заряда батареи

    # Отправка данных в Telemetry
    grid.telemetry.send({
        "position": position,
        "velocity": velocity,
        "altitude": altitude,
        "battery": battery
    })

    # Альтернативно: сохранение данных в Storage
    grid.storage.save("drone_status", {
        "position": position,
        "velocity": velocity,
        "altitude": altitude,
        "battery": battery
    })

    print("Состояние БПЛА отправлено в GRID.")

In [ ]:
# =============================== СТАРТУЕМ ===============================

# Запуск миссии
fly_mission()

# Отправка состояния БПЛА в GRID после завершения миссии
send_drone_status_to_grid(drone)


Взлёт...


🔍 Сканирование окружения...
🧱 Обнаружены препятствия: [(2, 2, 1)]
📡 Построение безопасного маршрута...
Начинаем движение по маршруту...
🛑 Обнаружено препятствие: [(2, 2, 1)]. Пересчитываю маршрут...


➡ Движение к точке: (0, 1, 1.0)


🛑 Обнаружено препятствие: [(2, 2, 1), (4, 4, 1)]. Пересчитываю маршрут...


➡ Движение к точке: (1, 1, 1.0)


...

➡ Движение к точке: (4, 5, 1.0)


🛑 Обнаружено препятствие: [(2, 2, 1), (4, 4, 1)]. Пересчитываю маршрут...


➡ Движение к точке: (5, 5, 1.0)


Посадка...


AttributeError: 'AirGenDrone' object has no attribute 'get_position'